## DBSCAN
- for each instance, DBSCAN counts # of instances are located within epsilon distance from it. This is called the instance's epsilon-neighbordhood
- In an instance has at least **min_samples** in its epsilon-neighborhood, then it is considered a core instance (located in dense regions)
- All instances in the neighborhood of a <u>core instance</u> belong to the same cluster. **Note: this can include other core instances**. So a sequence of core instances form a single cluster
- Any instance that is not a core instance and does not belong in a cluster (no core instance within epsilon distance nearby) is considered an anomaly

Algo works well if clusters are dense enough and separated by sparse regions

In [2]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [4]:
dbscan.labels_[:10]

array([ 0,  1,  2,  3,  2, -1,  0,  3,  1,  2])

instances that have labels of -1 are anomalies

In [11]:
dbscan.core_sample_indices_[:10] #indexes of core instances

array([ 0,  1,  2,  3,  4,  6,  9, 10, 12, 15])

In [9]:
dbscan.components_ # actual core instances

array([[ 0.54111778, -0.35570409],
       [ 0.61182902,  0.77550089],
       [ 1.52735624, -0.37381018],
       ...,
       [ 1.81509578, -0.05764226],
       [ 0.11110464,  0.08696671],
       [ 0.79458371,  0.62583022]])

DBSCAN does not have a predict() method because other classification algorithms can be better

In [13]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_]) # KNN classifier is trained on the core instances

KNeighborsClassifier(n_neighbors=50)

In [15]:
import numpy as np

X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
knn.predict(X_new)
knn.predict_proba(X_new)

array([[0.2 , 0.  , 0.  , 0.  , 0.  , 0.8 , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.46, 0.52, 0.  , 0.  , 0.  , 0.  , 0.02],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ]])

Problem: No anomalies were in the training set because the corecomponents were used to train the classifier so the classifier will always choose a cluster regardless of the distance
Solution: Introduce a max distance that will label a new instance as an anomaly if it exceed it

In [35]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1) # returns the distance and indices of the k nearest neighbors in the training set
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx] # the array of labels the new instances are closest to
y_pred[y_dist > 0.2] = -1
y_pred.ravel()

array([-1,  0,  6, -1])

## DBSCAN in a nutshell
- very simple yet powerful
- two hyperparameters (eps and min_samples)
- Can be impossible for it to capture all clusters if the density varies and gaps between clusters is too dense
- computational complexity: O(mlog(m))
    - Scikit's implementation requires O(m^2) if eps is large